In [305]:
%load_ext autoreload
%autoreload 2
import matplotlib as mpl
%matplotlib inline
import scipy as sp
import numpy as np
import pandas as pd
# sometimes order maters on these
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import oBehave.plotting_stuff as obps
import oBehave.helperfunctions as obhf
import oBehave.neural_analysis as obna
import oBehave.behavior_anaylsis as obba
import sklearn.decomposition as decomp


from visual_behavior.ophys.dataset.visual_behavior_ophys_dataset import VisualBehaviorOphysDataset
from visual_behavior.ophys.response_analysis.utilities import get_trace_around_timepoint,get_nearest_frame
from visual_behavior.ophys.response_analysis.response_analysis import ResponseAnalysis

#experiment_id = 639438856# experiment B
experiment_id = 639253368

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [307]:
manifest = obhf.load_manifest()
obba.includeNovelSession(manifest);
#dataset= VisualBehaviorOphysDataset(experiment_id=experiment_id, cache_dir=obhf.drive_path)
#analysis = ResponseAnalysis(dataset)

In [229]:
def computeEricsDimension(dataset,analysis):
    dataset= VisualBehaviorOphysDataset(experiment_id=rows.experiment_id, cache_dir=obhf.drive_path)
    analysis = ResponseAnalysis(dataset)
    num_trials = len(dataset.stimulus_table)
    fr = killnancells(dataset,analysis)
    use_cells = np.unique(fr.cell)
    response_array = np.empty((num_trials,len(use_cells)))
    for i in range(num_trials):
        response_array[i,:] = fr[fr.flash_number==i].mean_response.values
    PCA = decomp.PCA()
    X = PCA.fit(response_array-response_array.mean())
    D = 1/sum((PCA.explained_variance_ratio_)**2.)
    return D

In [316]:
def killnancells(dataset,analysis):
    bad_cell_indices = []
    bad_cell_ids = []
    for cell_index in range(dataset.dff_traces.shape[0]):
        if np.isnan(dataset.dff_traces[cell_index][0]): #check whether first datapoint is nan
           bad_cell_indices.append(cell_index)
           bad_cell_ids.append(dataset.get_cell_specimen_id_for_cell_index(cell_index))
    fdf = analysis.flash_response_df.copy()
    fdf = fdf[fdf.cell.isin(bad_cell_indices)==False]
    return fdf

In [318]:
keylist = ['experiment_id','session_type','targeted_structure','cre_line','dimension']
dim = {k:[]for k in keylist}
for ii, rows in manifest.iterrows():
    if rows.first_session:
        dataset= VisualBehaviorOphysDataset(experiment_id=rows.experiment_id, cache_dir=obhf.drive_path)
        analysis = ResponseAnalysis(dataset)
        num_trials = len(dataset.stimulus_table)
        fr = killnancells(dataset,analysis)
        use_cells = np.unique(fr.cell)
        response_array = np.empty((num_trials,len(use_cells)))
        for i in range(num_trials):
            response_array[i,:] = fr[fr.flash_number==i].mean_response.values
        PCA = decomp.PCA()
        X = PCA.fit(response_array-response_array.mean())
        D = 1/sum((PCA.explained_variance_ratio_)**2.)
        ##
        dim['experiment_id'].append(rows.experiment_id)
        dim['session_type'].append(rows.session_type)
        dim['targeted_structure'].append(rows.targeted_structure)
        dim['cre_line'].append(rows.cre_line)
        dim['dimension'].append(D)
        print(rows.session_type+','+ rows.cre_line  +','+rows.targeted_structure+ ' : ' + str(dim['dimension'][-1]))

loading trial response dataframe
loading flash response dataframe
behavior_session_A,Vip-IRES-Cre,VISp : 3.22914068049
loading trial response dataframe
loading flash response dataframe
behavior_session_B,Vip-IRES-Cre,VISp : 1.79559554164
loading trial response dataframe
loading flash response dataframe
behavior_session_C,Vip-IRES-Cre,VISp : 2.69973565741
loading trial response dataframe
loading flash response dataframe
behavior_session_D,Vip-IRES-Cre,VISp : 3.74252931569
loading trial response dataframe
loading flash response dataframe
behavior_session_A,Vip-IRES-Cre,VISp : 2.15108419538
loading trial response dataframe
loading flash response dataframe
behavior_session_B,Vip-IRES-Cre,VISp : 3.31502929996
loading trial response dataframe
loading flash response dataframe
behavior_session_C,Vip-IRES-Cre,VISp : 4.54282828751
loading trial response dataframe
loading flash response dataframe
behavior_session_D,Vip-IRES-Cre,VISp : 4.54961460061
loading trial response dataframe
loading flash r

In [319]:
DimFrame = pd.DataFrame(dim)

In [324]:
A = DimFrame[(DimFrame.targeted_structure=='VISp')&(DimFrame['cre_line']=='Slc17a7-IRES2-Cre')]

,cre_line,dimension,experiment_id,session_type,targeted_structure
8,Slc17a7-IRES2-Cre,5.360926,652844352,behavior_session_A,VISp
9,Slc17a7-IRES2-Cre,11.130728,653053906,behavior_session_C,VISp
10,Slc17a7-IRES2-Cre,11.427268,653123781,behavior_session_D,VISp
11,Slc17a7-IRES2-Cre,2.513765,639253368,behavior_session_A,VISp
12,Slc17a7-IRES2-Cre,7.325675,639438856,behavior_session_B,VISp
13,Slc17a7-IRES2-Cre,14.605912,639769395,behavior_session_C,VISp
14,Slc17a7-IRES2-Cre,17.400306,639932228,behavior_session_D,VISp
19,Slc17a7-IRES2-Cre,5.119192,670396087,behavior_session_A,VISp
20,Slc17a7-IRES2-Cre,18.592314,671152642,behavior_session_B,VISp
21,Slc17a7-IRES2-Cre,7.631782,672185644,behavior_session_D,VISp
